In [1]:
!wget https://github.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/raw/master/datasets/HTL.csv

--2021-01-28 21:19:16--  https://github.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/raw/master/datasets/HTL.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/HTL.csv [following]
--2021-01-28 21:19:16--  https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/HTL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15254112 (15M) [text/plain]
Saving to: ‘HTL.csv’

HTL.csv             100%[===================>]  14.55M  --.-KB/s    in 0.1s    

2021-01-28 21:19:16 (148 MB/s)

In [2]:
import re
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session

In [3]:

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [4]:

seed = 42

In [5]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

In [7]:
data = pd.read_csv('HTL.csv')
data.head()

,text,polarity
0,"المكان الذي يمكنك فيه مراجعة الذات والتفكر هو كوكروبيت، غانا.... \r\nثمة الكثير عند زيارة غانا. وعلى الرغم من الفقر الذي سوف تلاحظه على طريقك إلى بيغ ميلي باكيارد، وجدت أن الناس في غانا يملكون ثراء القلب حتى رغم العوز. بيغ ميلي باكيارد هو مكان يمكنني فيه مراجعة الذات والتفكر . التقيت أشخاصاً كانوا يعملون على مبادرات مختلفة. بيغ ميلي هو مكان يمكننا فيه تبادل الملاحظات والآراء. بيغ ميلي مليء بالسلام وهادئ للغاية . الطعام ممتاز جداً! الطهاة ممتازون! وبالإضافة إلى ذلك, لا بد من الإشارة إلى أن العمال الذين لا يكلون، ويحرصون على سعادة نزلائهم هم من يجعلون المكان مضيافاً. هناك خيبة أمل واحدة كبيرة!! رغم أن الفنون والأشغال اليدوية يمكن شراؤها في الموقع إلا أن الأسعار غالية!!! مؤخراً أزالت السلطات في كوكروبيت محلات التجار الواقعة خارج شاطئ بيغ ميلي. كانت منتجات التجار أرخص! لا يزال هناك محل تجاري ""متجر مايسترو للفن الإفريقي في كوكروبيت"" يعرض الفنون والأشغال اليدوية الفريدة والملابس واللوحات. اشتريت لوحات مذهلة لايمكنك أن تجدها في بيغ ميلي. عندما تزور بيغ ميلي من فضلك خذ لحظة للذهاب إلى الشاطئ وعلى يمينك ستجد ""متجر مايسترو للفن الإفريقي"" المذهل. لا تنخدع بأسعار التجار في بيغ ميلي باكيارد. استمتع بالموسيقى والناس! الرجاء أن تقضي وقتاً ممتعاً.",1
1,موقع رائع وحديقة رائعة ويستحق نجمةّ إضافية \r\nعلى الرغم من أن الغرف ليست فاخرة، فهي نظيفة جداً وتعمل بشكل مثالي. بالتأكيد هناك بعض التصرفات الفردية والجماعية، ولكن كان هناك تكييف هواء كبير وجيد ونوافذ مزدوجة جيدة مع شاشات. منطقة الحديقة تحتوى على 3 مطاعم بالإضافة إلى بار رياضي. تبلغ تكلفة الإنترنت اللاسلكي 1:50 دولار للساعة، ولا يوجد خصم على الفترات الأطول، لذلك اضطررت إلى إغلاق حاسوبي أكثر. إف و بي رخيص جداً. يبدو أن هناك العديد من الزوار الأجانب هناك. 85 دولار أمريكي لليلة بما في ذلك بوفيه إفطار بمستوى نجمة واحدة. إقامة أفضل بكثير مما اعتقدت، سنعود بالتأكيد. شملت الإقامات السابقة فندق الأفريقية ريجنت و الدخول بالمطار غرب فندق (خيارريجينت أفريكان وفندق إيربيورت ويست (خيار قوي لتلك المنطقة من المدينة). حركة المرور في أكرا مُصابة بالشلل، لذا قم بعقد اجتماعاتك في فندق بالوما - وهو معروف جيداً.,0
2,أسوأ فندق أقمت فيه على الإطلاق \r\nيستغرق تسجيل الوصول حوالي 30 دقيقة، باهظ الثمن، غرف قديمة جداً وقذرة، سجادة قذرة جداً... فى المجمل تجربة سيئة للغاية.\r\nمثل هذا الفندق سيكون في آسيا، نجمة واحدة، ويمكن أن أذهب في مايو.\r\n30 دولار أمريكي.\r\nالسعر في فندق نوفوتيل حوالي 250 دولار أمريكي\r\n,-1
3,بدون روح كأنه فندق ثلاثة نجوم \r\nبدون إدارة احترافية، فإن هذا الفندق يبدو كمركب ضائع في البحر.\r\nالغرف موزعة بطريقة عشوائية والإفطار هو نقانق باردة سخيفة ونفس السلطة القديمة في كل يوم.\r\nكان مكيف الهواء يثير الضوضاء في الغرفة ومن المستحيل إيقافه.\r\nولكن مركز صحي رائع\r\n,0
4,فندق جميل مع سوء الإدارة والخدمات. \r\nمن الخارج بدا مبشرًا، لكن خيبة الأمل بدأت في وقت الغداء، جودة الطعام سيئة سيئة سيئة، والحساء كان مزريًا جدًا، والفواكه جودتها سيئة (البكيخ كان متعفنًا، واو!). اشتكى زملائي من عدم وجود مناشف في غرفهم في حوالي الساعة 8 مساءً، والبعض لم تكن أسرّتهم مرتبة حتى الساعة 5 مساءً. تناولنا عشاء مكسيكيًا ومرة أخرى كانت قائمة الطعام مزرية. لن أقيم هنا أبدًا، على الأقل أتحدث عن نفسي!,-1


In [8]:
data = data[data.polarity.isin([-1, 1])]

In [9]:
data['cleaned_text'] = data.text.apply(clean_text)

In [10]:

data = data[data.cleaned_text != ""]
data.head(3)

,text,polarity,cleaned_text
0,"المكان الذي يمكنك فيه مراجعة الذات والتفكر هو كوكروبيت، غانا.... \r\nثمة الكثير عند زيارة غانا. وعلى الرغم من الفقر الذي سوف تلاحظه على طريقك إلى بيغ ميلي باكيارد، وجدت أن الناس في غانا يملكون ثراء القلب حتى رغم العوز. بيغ ميلي باكيارد هو مكان يمكنني فيه مراجعة الذات والتفكر . التقيت أشخاصاً كانوا يعملون على مبادرات مختلفة. بيغ ميلي هو مكان يمكننا فيه تبادل الملاحظات والآراء. بيغ ميلي مليء بالسلام وهادئ للغاية . الطعام ممتاز جداً! الطهاة ممتازون! وبالإضافة إلى ذلك, لا بد من الإشارة إلى أن العمال الذين لا يكلون، ويحرصون على سعادة نزلائهم هم من يجعلون المكان مضيافاً. هناك خيبة أمل واحدة كبيرة!! رغم أن الفنون والأشغال اليدوية يمكن شراؤها في الموقع إلا أن الأسعار غالية!!! مؤخراً أزالت السلطات في كوكروبيت محلات التجار الواقعة خارج شاطئ بيغ ميلي. كانت منتجات التجار أرخص! لا يزال هناك محل تجاري ""متجر مايسترو للفن الإفريقي في كوكروبيت"" يعرض الفنون والأشغال اليدوية الفريدة والملابس واللوحات. اشتريت لوحات مذهلة لايمكنك أن تجدها في بيغ ميلي. عندما تزور بيغ ميلي من فضلك خذ لحظة للذهاب إلى الشاطئ وعلى يمينك ستجد ""متجر مايسترو للفن الإفريقي"" المذهل. لا تنخدع بأسعار التجار في بيغ ميلي باكيارد. استمتع بالموسيقى والناس! الرجاء أن تقضي وقتاً ممتعاً.",1,المكان الذي يمكنك فيه مراجعه الذات والتفكر هو كوكروبيت غانا ثمه الكثير عند زياره غانا وعلي الرغم من الفقر الذي سوف تلاحظه علي طريقك الي بيغ ميلي باكيارد وجدت ان الناس في غانا يملكون ثراء القلب حتي رغم العوز بيغ ميلي باكيارد هو مكان يمكنني فيه مراجعه الذات والتفكر التقيت اشخاصا كانوا يعملون علي مبادرات مختلفه بيغ ميلي هو مكان يمكننا فيه تبادل الملاحظات والاراء بيغ ميلي مليء بالسلام وهادئ للغايه الطعام ممتاز جدا الطهاه ممتازون وبالاضافه الي ذلك لا بد من الاشاره الي ان العمال الذين لا يكلون ويحرصون علي سعاده نزلائهم هم من يجعلون المكان مضيافا هناك خيبه امل واحده كبيره رغم ان الفنون والاشغال اليدويه يمكن شراؤها في الموقع الا ان الاسعار غاليه مؤخرا ازالت السلطات في كوكروبيت محلات التجار الواقعه خارج شاطئ بيغ ميلي كانت منتجات التجار ارخص لا يزال هناك محل تجاري متجر مايسترو للفن الافريقي في كوكروبيت يعرض الفنون والاشغال اليدويه الفريده والملابس واللوحات اشتريت لوحات مذهله لايمكنك ان تجدها في بيغ ميلي عندما تزور بيغ ميلي من فضلك خذ لحظه للذهاب الي الشاطئ وعلي يمينك ستجد متجر مايسترو للفن الافريقي المذهل لا تنخدع باسعار التجار في بيغ ميلي باكيارد استمتع بالموسيقي والناس الرجاء ان تقضي وقتا ممتعا
2,أسوأ فندق أقمت فيه على الإطلاق \r\nيستغرق تسجيل الوصول حوالي 30 دقيقة، باهظ الثمن، غرف قديمة جداً وقذرة، سجادة قذرة جداً... فى المجمل تجربة سيئة للغاية.\r\nمثل هذا الفندق سيكون في آسيا، نجمة واحدة، ويمكن أن أذهب في مايو.\r\n30 دولار أمريكي.\r\nالسعر في فندق نوفوتيل حوالي 250 دولار أمريكي\r\n,-1,اسوا فندق اقمت فيه علي الاطلاق يستغرق تسجيل الوصول حوالي دقيقه باهظ الثمن غرف قديمه جدا وقذره سجاده قذره جدا في المجمل تجربه سيئه للغايه مثل هذا الفندق سيكون في اسيا نجمه واحده ويمكن ان اذهب في مايو دولار امريكي السعر في فندق نوفوتيل حوالي دولار امريكي
4,فندق جميل مع سوء الإدارة والخدمات. \r\nمن الخارج بدا مبشرًا، لكن خيبة الأمل بدأت في وقت الغداء، جودة الطعام سيئة سيئة سيئة، والحساء كان مزريًا جدًا، والفواكه جودتها سيئة (البكيخ كان متعفنًا، واو!). اشتكى زملائي من عدم وجود مناشف في غرفهم في حوالي الساعة 8 مساءً، والبعض لم تكن أسرّتهم مرتبة حتى الساعة 5 مساءً. تناولنا عشاء مكسيكيًا ومرة أخرى كانت قائمة الطعام مزرية. لن أقيم هنا أبدًا، على الأقل أتحدث عن نفسي!,-1,فندق جميل مع سوء الاداره والخدمات من الخارج بدا مبشرا لكن خيبه الامل بدات في وقت الغداء جوده الطعام سيئه سيئه سيئه والحساء كان مزريا جدا والفواكه جودتها سيئه البكيخ كان متعفنا واو اشتكي زملائي من عدم وجود مناشف في غرفهم في حوالي الساعه مساء والبعض لم تكن اسرتهم مرتبه حتي الساعه مساء تناولنا عشاء مكسيكيا ومره اخري كانت قائمه الطعام مزريه لن اقيم هنا ابدا علي الاقل اتحدث عن نفسي


In [12]:
min_sample = data.groupby(['polarity']).count().text.min()
input_data = pd.concat([data[data.polarity == 1].head(min_sample), 
                        data[data.polarity == -1].head(min_sample)])
input_data.groupby(['polarity']).count()

,text,cleaned_text
polarity,,
-1,2645,2645
1,2645,2645


In [13]:
X = input_data.cleaned_text.values

In [14]:

Y = np.asarray(input_data.polarity.values).astype('float32')
Y = Y.clip(0, 1)

In [15]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

vocab_size = len(tokenizer.word_index) + 1
print("vocab size:", vocab_size)

vocab size: 48682


In [16]:
maxlen = 300
X = pad_sequences(X, padding='post', maxlen=maxlen)

In [17]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [18]:
X_train, X_test, label_train, label_test = train_test_split(X, Y, test_size=0.3,
                                                            random_state=seed)

In [19]:
print("Training:", len(X_train), len(label_train))
print("Testing: ", len(X_test), len(label_test))

Training: 3703 3703
Testing:  1587 1587


In [20]:
embedding_dim = 100
dropout = 0.5
opt = 'adam'
clear_session()

model = Sequential()
model.add(layers.Embedding(input_dim=num_words, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(100, dropout=dropout, 
                                           recurrent_dropout=dropout, 
                                           return_sequences=True)))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 300, 200)          160800    
_________________________________________________________________
global_max_pooling1d (Global (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               25728     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0

In [21]:
history = model.fit(X_train, label_train,
                    epochs=4,
                    verbose=True,
                    validation_data=(X_test, label_test),
                    batch_size=64)
loss, accuracy = model.evaluate(X_train, label_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss_val, accuracy_val = model.evaluate(X_test, label_test, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy_val))

Epoch 1/4
58/58 [==============================] - 145s 2s/step - loss: 0.6929 - accuracy: 0.5252 - val_loss: 0.6881 - val_accuracy: 0.5444
Epoch 2/4
58/58 [==============================] - 139s 2s/step - loss: 0.6219 - accuracy: 0.6540 - val_loss: 0.2619 - val_accuracy: 0.8891
Epoch 3/4
58/58 [==============================] - 140s 2s/step - loss: 0.2797 - accuracy: 0.9078 - val_loss: 0.2249 - val_accuracy: 0.9219
Epoch 4/4
116/116 [==============================] - 20s 172ms/step - loss: 0.0797 - accuracy: 0.9789
Training Accuracy: 0.9789
50/50 [==============================] - 7s 149ms/step - loss: 0.2025 - accuracy: 0.9193
Testing Accuracy:  0.9193


In [24]:
blind_sample = 200
blind_test = pd.concat([data[data.polarity == 1].tail(blind_sample), 
                        data[data.polarity == -1].tail(blind_sample)])

In [25]:
X_blind = blind_test.cleaned_text.values
Y_blind = np.asarray(blind_test.polarity.values).astype('float32')
Y_blind = Y_blind.clip(0, 1)

In [26]:
X_blind = tokenizer.texts_to_sequences(X_blind)
X_blind = pad_sequences(X_blind, padding='post', maxlen=maxlen)

In [27]:
pred_blind = model.predict(X_blind, verbose=True)

13/13 [==============================] - 3s 163ms/step


In [28]:
df_blind = pd.DataFrame({'REAL': Y_blind, 
                         'PRED': pred_blind.reshape(pred_blind.shape[0],), 
                         'TEXT': blind_test.cleaned_text})
df_blind = df_blind.reset_index()[['REAL', 'PRED', 'TEXT']]
df_blind.PRED = df_blind.PRED.round()
error_records = df_blind[df_blind.REAL != df_blind.PRED]

In [29]:

print("Number of misclassified reviews: {} out of {}".format(error_records.shape[0], df_blind.shape[0]))
print("Blind Test Accuracy:  {:.4f}".format(accuracy_score(df_blind.REAL, df_blind.PRED)))

Number of misclassified reviews: 56 out of 400
Blind Test Accuracy:  0.8600


In [31]:
df_blind.sample(n=3)

,REAL,PRED,TEXT
137,1.0,0.0,موقع رائع ما يقرب من الكمال لقد قضيت رحله عمل قصيره في فندق ماريوت مع غرفتين محجوزتين وكان افضل شيء هو موقع الفندق يقع علي يمين طريق اوركارد ونزلت طابقا واحدا سيرا علي الاقدام للوصول الي مركز للتسوق ويقع مركز ايون الشهير في الجهه المقابله ويمكنك المشي اليه داخل ممر مغطي من الفندق من السهل للوصول الي القطار الطعام في ماريوت كافيه جيد بشكل عام ومادبه الافطار جيده وقد استمتعت بالخدمه الجيده هناك ايضا كانت الغرف فسيحه ونظيفه ولديها تصميم حديث بسيط كان السرير مريحا وكان هناك مساحه كافيه في خزانه الملابس اذا كنت تنوي البقاء لفتره طويله وكان الحمام فسيحا وكذلك يوجد به حوضين بالاضافه الي منطقه مرحاض صغيره حصلت علي لعبه لينه صغيره علي شكل نمر اعطاني اياها خدمه تنظيف الغرف وقد اعجبتني فكره توزيع لعبه لينه صغيره كهديه تذكاريه ومع ذلك فقد كانت الشكوي الوحيده لكل واحد منا انا وزميلي هي منطقه الاستحمام فمنطقه الاستحمام لم يتم المحافظه عليها بشكل جيد بالنسبه لزميلي كانت منطقه الاستحمام سيئه وقد تدفقت المياه خارجها فيما عدا ذلك فقد حظينا باقامه مريحه في فندق ماريوت
8,1.0,1.0,روعه التعامل روعه التعامل هي عنوان الجوده وهذا يقودني الي ان اقول ان التاكا من اروع الفنادق السودانيه وفيه كل سبل الراحه ويمتاز بالموقع الجميل ولكن عدم موقف السيارات الكافي قد يؤدي الي صرف نظر الناس عنه ولذلك يجب عمل موقف سيارات لكي يكون من الدرجه الرفيعه
283,0.0,0.0,لا تقم هنا اجراءات الدخول كانت في الرابعه لذلك تركنا امتعتنا في تلك الغرفه الا انه كان علينا دفع دولارين من اجل ذلك التواليت كان يسرب المياه ولم يهتم الموظفون اطلاقا باصلاحه مناشف الشاطيء التي حصلنا عليها من مكتب الاستقبال كانت مقرفه لم يتم تغير ملاءات السرير ولم يتم توفير مناشف وقطع قماش كافيه كان علي ان احضرها بنفسي من مكتب الاستقبال غادرت المكان وانا اعاني من العطس ورشح في الانف وربما كان ذلك بسبب العفن كان لدينا خمس قنوات فقط علي التلفاز لا يوجد اضاءه عند المراه لقد كان شيئا فظيعا


In [32]:
with open('model_acc{}.json'.format(round(accuracy_val, 4)), 'w') as f:
    f.write(model.to_json())
    f.close()

In [33]:
model.save_weights('model_acc{}.h5'.format(round(accuracy_val, 4)))